# 소설 작성 

## skt/kogpt2-base-v2

In [ ]:
!pip install --upgrade pip
!pip install numpy==1.26.4
!pip install torch==2.2.2 --index-url https://download.pytorch.org/whl/cpu
!pip install transformers==4.41.2


In [8]:
# =========================================
# 1) 로딩
# =========================================
import re, random, textwrap
from typing import List, Tuple

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_NAME = "skt/kogpt2-base-v2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model     = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# GPT-2 계열은 pad 토큰이 없으므로 eos를 pad로 사용
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model.to(DEVICE)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [9]:
# =========================================
# 2) 유틸: 문장 경계/키워드 체크/연결어 등
# =========================================
CONNECTIVES = ["그러나", "한편", "그때", "곧", "결국", "마침내", "한동안", "이윽고", "그럼에도", "하지만"]

def split_sentences_kr(text: str) -> List[str]:
    sents = re.split(r'(?<=[\.!?])\s+', text.strip())
    sents = [s.strip() for s in sents if s.strip()]
    return sents

def join_to_range(sents: List[str], min_n=5, max_n=7) -> str:
    if len(sents) < min_n:
        # 짧으면 마지막 문장을 약간 늘리는 방식으로 보정(그냥 이어 붙이기)
        return " ".join(sents)
    return " ".join(sents[:min(len(sents), max_n)])

def ensure_connectives(sents: List[str]) -> List[str]:
    # 2~4번째 문장에 연결어가 없으면 부드럽게 하나 추가
    for idx in range(1, min(len(sents), 5)):
        if not any(sents[idx].startswith(c) for c in CONNECTIVES):
            sents[idx] = f"{random.choice(CONNECTIVES)} " + sents[idx]
    return sents

def contains_all_keywords(text: str, keywords: List[str]) -> bool:
    low = text.lower()
    return all(kw.lower() in low for kw in keywords)


In [10]:
# =========================================
# 3) 프롬프트 설계 (계획 → 집필)
# =========================================
def build_outline_prompt(keywords: List[str]) -> str:
    kw = ", ".join(keywords)
    return (
        "아래 키워드들을 모두 중심 motief로 삼아 짧은 소설의 개요를 4줄로 작성하세요.\n"
        "각 줄은 항목명: 간결한 구절 형태로 쓰고 군더더기를 피하세요.\n"
        "항목: 배경, 인물, 갈등, 해결\n"
        f"키워드: {kw}\n"
        "개요:\n"
        "- 배경:\n- 인물:\n- 갈등:\n- 해결:"
    )

def build_story_prompt(keywords: List[str], outline: str, tone: str = "담백하고 서정적인 3인칭 시점"):
    kw = ", ".join(keywords)
    return (
        "다음 개요를 바탕으로 한국어 짧은 소설을 5~7문장으로 작성하세요.\n"
        "문장은 자연스럽고 연결이 매끄럽게 이어지며, 과도한 반복을 피합니다.\n"
        f"문체는 {tone}으로 유지하고, 모든 키워드를 반드시 포함합니다.\n"
        "문장마다 마침표로 끝맺고, 이야기의 호흡이 부드럽게 흐르도록 하세요.\n\n"
        f"[키워드] {kw}\n"
        f"[개요]\n{outline}\n"
        "[소설]"
    )


In [11]:
# =========================================
# 4) 생성기 (대조 탐색 우선, 폴백: 빔)
# =========================================
@torch.inference_mode()
def generate_text(prompt: str,
                  strategy: str = "contrastive",
                  max_new_tokens: int = 220,
                  temperature: float = 0.9,
                  top_p: float = 0.92,
                  num_beams: int = 5,
                  penalty_alpha: float = 0.6,  # contrastive
                  top_k: int = 4,             # contrastive
                  repetition_penalty: float = 1.12,
                  no_repeat_ngram_size: int = 3) -> str:

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(DEVICE)

    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        repetition_penalty=repetition_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

    if strategy == "contrastive":
        # Contrastive Search: 더 일관적인 문장 경향
        try:
            outputs = model.generate(
                input_ids,
                do_sample=False,
                penalty_alpha=penalty_alpha,
                top_k=top_k,
                **gen_kwargs
            )
        except Exception:
            # 일부 환경/버전에서 contrastive 미지원 → 빔서치로 대체
            outputs = model.generate(
                input_ids,
                do_sample=False,
                num_beams=num_beams,
                length_penalty=1.05,
                early_stopping=True,
                **gen_kwargs
            )
    elif strategy == "beam":
        outputs = model.generate(
            input_ids,
            do_sample=False,
            num_beams=num_beams,
            length_penalty=1.05,
            early_stopping=True,
            **gen_kwargs
        )
    else:  # sampling
        outputs = model.generate(
            input_ids,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            **gen_kwargs
        )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # 프롬프트 이후만 추출
    return text[len(prompt):].strip()


In [12]:
# =========================================
# 5) 엔드투엔드: 키워드 → 개요 → 소설 → 후편집(+재시도)
# =========================================
def generate_outline(keywords: List[str],
                     strategy="beam",
                     max_new_tokens=120) -> str:
    prompt = build_outline_prompt(keywords)
    raw = generate_text(prompt, strategy=strategy, max_new_tokens=max_new_tokens)
    # 간단 정리: 불릿 없으면 줄 단위로 보정
    lines = [ln.strip(" -") for ln in raw.splitlines() if ln.strip()]
    # 첫 4줄만 사용
    cleaned = []
    for ln in lines:
        if any(h in ln for h in ["배경", "인물", "갈등", "해결"]):
            cleaned.append(ln)
        elif len(cleaned) < 4:
            cleaned.append(ln)
        if len(cleaned) == 4:
            break
    # 포맷이 흐트러져도 그대로 사용
    return "\n".join(f"- {ln}" for ln in cleaned[:4])

def post_edit_story(text: str, min_s=5, max_s=7) -> str:
    sents = split_sentences_kr(text)
    sents = [re.sub(r"\s+", " ", s).strip() for s in sents if s.strip()]
    sents = ensure_connectives(sents)
    story = join_to_range(sents, min_n=min_s, max_n=max_s)
    return story

def generate_short_story_smooth(
    keywords: List[str],
    tone: str = "담백하고 서정적인 3인칭 시점",
    strategy: str = "contrastive",          # "contrastive" | "beam" | "sample"
    max_new_tokens_outline: int = 120,
    max_new_tokens_story: int = 220,
    max_retries: int = 3,
    seed: int = 42
) -> Tuple[str, dict]:

    assert len(keywords) > 0, "키워드를 한 개 이상 입력하세요."
    random.seed(seed); torch.manual_seed(seed)

    # 1) 개요 생성
    outline = generate_outline(keywords, strategy="beam", max_new_tokens=max_new_tokens_outline)

    # 2) 소설 생성 (+재시도: 키워드 누락 시 톤/전략 고정, 페널티만 조금씩 조정)
    rep = 1.10
    for attempt in range(1, max_retries+1):
        prompt_story = build_story_prompt(keywords, outline, tone=tone)
        raw = generate_text(
            prompt_story,
            strategy=strategy,
            max_new_tokens=max_new_tokens_story,
            repetition_penalty=rep,
            no_repeat_ngram_size=3
        )
        story = post_edit_story(raw, min_s=5, max_s=7)

        if contains_all_keywords(story, keywords):
            return story, {"attempt": attempt, "strategy": strategy, "repetition_penalty": rep, "tone": tone}

        # 키워드 빠졌으면 재시도(반복 페널티↑)
        rep = min(1.25, rep + 0.05)

    # 3) 마지막 시도 결과라도 반환
    return story, {"attempt": attempt, "strategy": strategy, "repetition_penalty": rep, "tone": tone}


In [13]:
# =========================================
# 6) 테스트
# =========================================
cases = [
    ["비", "소녀", "들판", "우산"],
    ["바다", "라디오", "여름밤"],
    ["연구실", "노트", "낡은 시계", "비밀"]
]

for kws in cases:
    story, meta = generate_short_story_smooth(
        kws,
        tone="담백하고 서정적인 3인칭 시점",   # 다른 톤 예: "차분한 1인칭 회고체", "미묘하게 긴장감 있는 묘사체"
        strategy="contrastive",              # contrastive ↔ beam ↔ sample
        max_new_tokens_outline=120,
        max_new_tokens_story=220,
        max_retries=3,
        seed=42
    )
    print(f"\n=== 키워드: {kws} ===")
    print(textwrap.fill(story, width=90))
    print("메타:", meta)



=== 키워드: ['비', '소녀', '들판', '우산'] ===
비, 소녀,들판, 우산 이 게임의 주인공. 한편 성우는 카와스미 아야코/정규 1기 오프닝에서 나왔던 오리지널 캐릭터. 그러나 원작과 같은 인물로, 마법소녀 리리컬
나노하 일레븐에 등장하는 괴인 중 한 명이다. 결국 본래 이름은 코바야시 히토미의 친누나이자 사쿠라이 유우키가 맡았다. 곧 고스트 버스터즈의 히로인으로 등장할
예정이었으나 갑작스럽게 불발된 바 있다. 그런데 이 캐릭터의 외모를 보고 팬들이 놀라는 일이 벌어지기도 했다. (...
메타: {'attempt': 1, 'strategy': 'contrastive', 'repetition_penalty': 1.1, 'tone': '담백하고 서정적인 3인칭 시점'}

=== 키워드: ['바다', '라디오', '여름밤'] ===
해리포터 시리즈와 마블 코믹스의 캐릭터를 원작으로 한 영화. 한편 이 게임은 전작과 같은 난이도를 자랑한다. 하지만 다행히 스토리가 길기 때문에 플레이어들의
평타에 맞출 수 있는 스킬들이 꽤 많이 나오며, 그 중 가장 높은 점수를 받을 수 있다는 점이 매력 포인트. 한동안 또한 스테이지 클리어 보상으로 아이템 획득량이
증가하기 때문에, 초반에는 어렵지 않게 클리어가 가능하다. 그러나 고레벨 유저들을 위한 패치도 진행중이다. 전작의 보스 몬스터인 카미카제 일행을 처치하면 레벨
5가 되는 것을 볼 수 있다. 보스몬스터로 등장했을 때 얻을 수 있었던 카드들은 다음과 같다.
메타: {'attempt': 3, 'strategy': 'contrastive', 'repetition_penalty': 1.25, 'tone': '담백하고 서정적인 3인칭 시점'}

=== 키워드: ['연구실', '노트', '낡은 시계', '비밀'] ===
완성도 높은 시나리오가 나올 수 있도록 도와줍니다. 한편 (단순 암기하는 방식 ) ※ 이 글을 읽다 보면 다음과 같은 내용이 떠오릅니다. 하지만 2016년 1월,
한화생명은 서울 여의도 63빌딩에서 ‘생